In [6]:
import numpy as np
from scipy.sparse import diags, kron, eye, linalg
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib qt

# Coefficients
Lx = 12
Ly = 5
T_ext = 25

# A
N = 120
h = Lx / N
M = int(Ly / h)

def F_func(x, y):
    return 100 * np.exp(-1/2 * (x - 4)**2 - 4 * (y - 1)**2)

x = np.arange(h, Lx, h)
y = np.arange(h, Ly, h)
X, Y = np.meshgrid(x, y)
F = F_func(X, Y).T


In [7]:

# Create the sparse matrices Sx and Sy
Sx = (1 / h**2) * diags([1, -2, 1], [-1, 0, 1], shape=(N-1, N-1),format='csc')
Sy = (1 / h**2) * diags([1, -2, 1], [-1, 0, 1], shape=(M-1, M-1), format='csc')

# Apply boundary conditions
Sx[0, 0] = -2 / (3 * h**2)
Sx[0, 1] = 2 / (3 * h**2)
Sx[-1, -1] = -2 / (3 * h**2)
Sx[-1, -2] = 2 / (3 * h**2)

Sy[-1, -1] = -2 / (3 * h**2)
Sy[-1, -2] = 2 / (3 * h**2)

# Create the operator A using Kronecker products
A = kron(eye(M-1), Sx, format='csc') + kron(Sy, eye(N-1), format='csc')

F[:, 0] = F[:, 0] + T_ext / h**2
f = F.reshape(((N-1) * (M-1), 1), order='F')

dt = 0.1
T = 40

u0 = T_ext * np.ones(f.shape)

# Crank Nicolson
tau = np.arange(0, T, dt)
saved_u = np.zeros((len(u0), len(tau)))
saved_u[:, 0] = u0[:, 0]

uk = u0

I = eye(len(u0), format='csc')

for i in range(len(tau)):
    I = eye(len(uk), format='csc')
    A_half = I - 0.5 * dt * A
    b = (I + 0.5 * dt * A).dot(uk) + dt * f
    u_new = spsolve(A_half, b)
    saved_u[:, i] = u_new
    uk = np.array([u_new]).T

u_62 = np.zeros(len(tau))

# Create a figure and 3D axis for the animation
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Function to update the 3D plot for each frame
def update(frame):
    ax.clear()
    u = saved_u[:, frame].reshape(N - 1, M - 1, order='F')
    u_y0 = T_ext * np.ones(N - 1)
    u_M = 1/3 * (4 * u[:, -1] - u[:, -2])
    u = np.column_stack((u_y0, u, u_M))
    u_N = 1/3 * (4 * u[-1, :] - u[-2, :])
    u_x0 = 1/3 * (4 * u[0, :] - u[1, :])
    u = np.row_stack((u_x0, u, u_N))

    x, y = np.meshgrid(np.arange(0, Lx + h, h), np.arange(0, Ly + h, h))
    x, y = x.T, y.T  # Transpose x and y to match the shape of u
    ax.plot_surface(x, y, u, cmap='viridis')
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("Temperature")
    ax.set_zlim(25,70)
    ax.set_xlim(0,Lx)
    ax.set_ylim(0,Ly)
    ax.set_title(f'Time {np.round(frame*dt,2)+dt} seconds')

    #Save u(6,2)
    u_62[frame] = u[np.round(6/h).astype(int), np.round(2/h).astype(int)]

ani = FuncAnimation(fig, update, frames=range(int(T / dt)), repeat=False)


In [3]:

# C
print(f"u(6,2,40) = {u_62[-1]:.4f}")

u(6,2,40) = 0.0000
